<a href="https://colab.research.google.com/github/luciamartinf/BigData/blob/main/CovNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CovNet Model
Ángela Gómez Sacristán, Álvaro González Berdasco y Lucía Martín Fernández

## Import libraries


In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os, random
from pathlib import Path
import cv2

import sklearn
from sklearn.metrics import accuracy_score, classification_report

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import load_img
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing import image

import warnings
warnings.filterwarnings('ignore')

## Load and Transform data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Define path to the data directory
dir_alldata = Path('/content/drive/MyDrive/Colab Notebooks/chest_xray')
# Path to train directory (Fancy pathlib...no more os.path!!)
train_data_dir = dir_alldata / 'train'

# Path to validation directory
validation_data_dir = dir_alldata / 'val'

# Path to test directory
test_data_dir = dir_alldata / 'test'

# Get the path to the normal and pneumonia sub-directories
normal_cases_train = train_data_dir / 'NORMAL'
pneumonia_cases_train = train_data_dir / 'PNEUMONIA'

In [ ]:
img_width, img_height = 150,150
nb_train_sample =1000
nb_validation_samples =100

epochs = 50
batch_size = 20

if K.image_data_format()=="channels_first":
    input_shape =(3,img_width, img_height)
else:
    input_shape =(img_width, img_height,3)

train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

validation_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator= train_datagen.flow_from_directory(train_data_dir, target_size =(150,150),batch_size=batch_size, class_mode="binary" )

validation_generator = validation_datagen.flow_from_directory(validation_data_dir, target_size = (150,150),batch_size=batch_size, class_mode="binary")
 
test_generator = test_datagen.flow_from_directory(test_data_dir, target_size = (150,150),batch_size=batch_size, class_mode="binary")
 

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


## CovNet Model

### Building model

In [ ]:
# Initialising the convnet and defining it by adding layers
model = Sequential()
model.add(Conv2D(32,(3,3),input_shape=input_shape))
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(32,(3,3)))
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3)))
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation("sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 activation (Activation)     (None, 148, 148, 32)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 72, 72, 32)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 32)       0         
 2D)                                                    

### Compiling model

In [ ]:
# Compiling the Convnet
model.compile(loss= "binary_crossentropy", optimizer="adam", metrics=["accuracy"])

### Fitting model

In [ ]:
# Train de convnet
history = model.fit_generator(train_generator, steps_per_epoch=nb_train_sample//batch_size, epochs=epochs,validation_data=validation_generator, validation_steps=nb_validation_samples//batch_size)

Epoch 1/50
50/50 [==============================] - ETA: 0s - loss: 0.5972 - accuracy: 0.7460

50/50 [==============================] - 196s 4s/step - loss: 0.5972 - accuracy: 0.7460 - val_loss: 0.8403 - val_accuracy: 0.5000
Epoch 2/50
50/50 [==============================] - 127s 3s/step - loss: 0.4278 - accuracy: 0.7890
Epoch 3/50
50/50 [==============================] - 104s 2s/step - loss: 0.3627 - accuracy: 0.8112
Epoch 4/50
50/50 [==============================] - 99s 2s/step - loss: 0.3494 - accuracy: 0.8450
Epoch 5/50
50/50 [==============================] - 83s 2s/step - loss: 0.2828 - accuracy: 0.8785
Epoch 6/50
50/50 [==============================] - 74s 1s/step - loss: 0.2699 - accuracy: 0.8980
Epoch 7/50
50/50 [==============================] - 68s 1s/step - loss: 0.2435 - accuracy: 0.9020
Epoch 8/50
50/50 [==============================] - 61s 1s/step - loss: 0.2660 - accuracy: 0.8876
Epoch 9/50
50/50 [==============================] - 56s 1s/step - loss: 0.2488 - accuracy: 0.8840
Epoch 10/50
50/50 [==============================] - 53s 1s/step - loss: 0.2535 - ac

### Model evaluation 

In [ ]:
#Testing on the test set
test_evaluation = model.evaluate_generator(test_generator)

In [ ]:
# Evaluating Test accuracy
print('The accuracy on test set :',test_evaluation[1]*100 )

The accuracy on test set : 84.77563858032227


The results of this model are very good (84.8% accuracy). 